In [156]:
import nltk
import nltk.corpus
import nltk.tokenize.punkt
import nltk.stem.snowball
from nltk import word_tokenize
from nltk.corpus import wordnet, cess_esp

import string
import pickle

In [157]:
stopwords = nltk.corpus.stopwords.words()
stopwords.extend(string.punctuation)
stopwords.append('')

In [188]:
#nltk.download('cess_esp')
nltk.download('all')

import nltk
from nltk.corpus import cess_esp
from nltk.tag import TrigramTagger, UnigramTagger
from nltk.tag.perceptron import PerceptronTagger

cess_esp = cess_esp.tagged_sents()


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [204]:
class BackoffTagger:
    def __init__(self):
        self._taggers = [PerceptronTagger()]
        
class TexApproximation:
    
    def __init__(self, text, compared_text):
        self.text = text
        self.compared_text = compared_text
        self.lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    
    def get_wordnet_pos(self, pos_tag):
        print('>>>>>', pos_tag[1], pos_tag[0])
        if pos_tag[1].startswith('J'):
            return (pos_tag[0], wordnet.ADJ)
        elif pos_tag[1].startswith('V'):
            return (pos_tag[0], wordnet.VERB)
        elif pos_tag[1].startswith('N'):
            return (pos_tag[0], wordnet.NOUN)
        elif pos_tag[1].startswith('R'):
            return (pos_tag[0], wordnet.ADV)
        else:
            return (pos_tag[0], wordnet.NOUN)
    
    def is_similar(self, threshold=0.5):
        tagged_sents = nltk.corpus.cess_esp.tagged_sents()
        spanish_text = nltk.corpus.cess_esp.tagged_sents()
        
        train_sents = spanish_text[:3000]
        test_sents = spanish_text[3000:]
        
        t0 = nltk.DefaultTagger('nc')
        affix_tagger = nltk.AffixTagger(tagged_sents, backoff=t0)
        tagger4 = UnigramTagger(train_sents, cutoff=0, backoff=affix_tagger)
        print ("Trigram ", tagger4.evaluate(test_sents))
        
        #print('>>>>', tagger4.tag(word_tokenize(self.text)))
        
        for word, tag in tagger4.tag(word_tokenize(self.text)):
            print(word, '->', tag)

        
        
        pos_a = map(self.get_wordnet_pos, nltk.pos_tag(word_tokenize(self.text)))
        pos_b = map(self.get_wordnet_pos, nltk.pos_tag(word_tokenize(self.compared_text)))
        
        print('???', pos_a, pos_b)
        
        lemmae_a = [self.lemmatizer.lemmatize(token.lower().strip(string.punctuation), pos) for token, pos in pos_a \
                        if pos == wordnet.NOUN and token.lower().strip(string.punctuation) not in stopwords]
        lemmae_b = [self.lemmatizer.lemmatize(token.lower().strip(string.punctuation), pos) for token, pos in pos_b \
                        if pos == wordnet.NOUN and token.lower().strip(string.punctuation) not in stopwords]
        # Calculate Jaccard similarity
        ratio = len(set(lemmae_a).intersection(lemmae_b)) / float(len(set(lemmae_a).union(lemmae_b)))
        print(lemmae_a, lemmae_b, ratio)
        return (ratio >= threshold)
    
    def is_ci_token_stopword_set_match(self, threshold=0.5):
        tokens_a = [token.lower().strip(string.punctuation) for token in word_tokenize(self.text) \
                        if token.lower().strip(string.punctuation) not in stopwords]
        tokens_b = [token.lower().strip(string.punctuation) for token in word_tokenize(self.compared_text) \
                        if token.lower().strip(string.punctuation) not in stopwords]

        # Calculate Jaccard similarity
        ratio = len(set(tokens_a).intersection(tokens_b)) / float(len(set(tokens_a).union(tokens_b)))
        return (ratio >= threshold)


In [205]:
ta = TexApproximation('una persona puede hacer una denuncia anónimamente?', #a person can make a complaint anonymously?
                      'people could complaint anonymously?') #las personas pueden denunciar anónimamente?
if ta.is_similar(0.5):
    print('Las preguntas son parecidas')
else:
    print('Las preguntas no tienen nada que ver...')
#tokenizer = word_tokenize("Tokenize me")
#print(tokenizer)

Trigram  0.874055543562458
una -> di0fs0
persona -> ncfs000
puede -> vmip3s0
hacer -> vmn0000
una -> di0fs0
denuncia -> ncfs000
anónimamente -> rg
? -> Fit
??? <map object at 0x7ff8c5068310> <map object at 0x7ff8c5068280>
>>>>> JJ una
>>>>> NN persona
>>>>> NN puede
>>>>> NN hacer
>>>>> JJ una
>>>>> NN denuncia
>>>>> NN anónimamente
>>>>> . ?
>>>>> NNS people
>>>>> MD could
>>>>> VB complaint
>>>>> RB anonymously
>>>>> . ?
['persona', 'puede', 'hacer', 'denuncia', 'anónimamente'] ['people', 'could'] 0.0
Las preguntas no tienen nada que ver...
